In [1]:

import pandas as pd
import random
import seaborn as sns
from scipy import stats
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, RobustScaler,OrdinalEncoder
import pandas as pd
from pathlib import Path
from projimmo.data import *
from projimmo.preprocessor import *

query = f"""
        SELECT *
        FROM `{GCP_PROJECT}`.{BQ_DATASET}.DVF_{DATA_YEAR}
    """

    # $CHA_BEGIN
    # Retrieve data using `get_data_with_cache`
#data_query_cache_path = Path(LOCAL_DATA_PATH).joinpath("raw", f"query_{min_date}_{max_date}_{DATA_SIZE}.csv")
 #   data_query = get_data_with_cache(
 #       query=query,
 #       gcp_project=GCP_PROJECT,
 #       cache_path=data_query_cache_path,
 #       data_has_header=True
 #   ).joinpath("raw", f"query_{min_date}_{max_date}_{DATA_SIZE}.csv")
data_query_cache_path=Path(f"raw_data/valeursfoncieres-2023.txt")
# Lire le fichier entier
#df = pd.read_csv("raw_data/valeursfoncieres-2023.txt", sep="|", dtype=str)
print(data_query_cache_path)
df= get_data_with_cache(query=query,
gcp_project=GCP_PROJECT,
        cache_path=data_query_cache_path,
        data_has_header=True
   )


#load_data_to_bq(
#        data_processed_with_timestamp,
 #       gcp_project=GCP_PROJECT,
 #       bq_dataset=BQ_DATASET,
 #       table=f'processed_{DATA_SIZE}',
 #       truncate=True
  #  )

df_0=clean_data(df)

df.dtypes
df_1=clean_outliers(df_0)
#load_data_to_bq(
#        df,
 #       gcp_project=GCP_PROJECT,
#        bq_dataset=BQ_DATASET,
 #       table=f'clean_data_{DATA_YEAR}',
 #       truncate=True
 #   )

df_2=preprocess_features(df_1)


raw_data/valeursfoncieres-2023.txt

Load data from local CSV...
✅ Data loaded, with shape (3727005, 43)
✅ X_processed, with shape (74729, 41)


In [2]:
df_2.columns

Index(['surface_reelle_bati', 'surface_terrain', 'somme_surface_carrez',
       'code_type_local_1', 'code_type_local_2', 'type_de_voie_ALL',
       'type_de_voie_AUTRE', 'type_de_voie_AV', 'type_de_voie_BD',
       'type_de_voie_CHE', 'type_de_voie_CRS', 'type_de_voie_IMP',
       'type_de_voie_PL', 'type_de_voie_RES', 'type_de_voie_RTE',
       'type_de_voie_RUE', 'month_mutation_1', 'month_mutation_2',
       'month_mutation_3', 'month_mutation_4', 'month_mutation_5',
       'month_mutation_6', 'month_mutation_7', 'month_mutation_8',
       'month_mutation_9', 'month_mutation_10', 'month_mutation_11',
       'month_mutation_12', 'year_mutation_2023', 'departement_13',
       'departement_31', 'departement_33', 'departement_34', 'departement_44',
       'departement_59', 'departement_67', 'departement_69', 'departement_75',
       'nombre_pieces_principales', 'valeur_fonciere', 'nombre_de_lots'],
      dtype='object')

In [2]:
df_2.nunique()
    # Supposons que les colonnes 'date_mutation' et 'valeur_fonciere' sont uniques
unique_keys = df_1[['year_mutation','month_mutation','valeur_fonciere']].to_dict('records')
len(unique_keys), df_2.shape,unique_keys

(74729,
 (74729, 41),
 [{'year_mutation': 2023,
   'month_mutation': 1,
   'valeur_fonciere': 12.345838935721968},
  {'year_mutation': 2023,
   'month_mutation': 1,
   'valeur_fonciere': 12.409017571151187},
  {'year_mutation': 2023,
   'month_mutation': 1,
   'valeur_fonciere': 11.37940550085183},
  {'year_mutation': 2023,
   'month_mutation': 1,
   'valeur_fonciere': 11.813037464800539},
  {'year_mutation': 2023,
   'month_mutation': 2,
   'valeur_fonciere': 11.849404844423074},
  {'year_mutation': 2023,
   'month_mutation': 1,
   'valeur_fonciere': 11.951186847493476},
  {'year_mutation': 2023,
   'month_mutation': 1,
   'valeur_fonciere': 11.289794413577894},
  {'year_mutation': 2023,
   'month_mutation': 1,
   'valeur_fonciere': 12.345838935721968},
  {'year_mutation': 2023,
   'month_mutation': 1,
   'valeur_fonciere': 11.407576060361786},
  {'year_mutation': 2023,
   'month_mutation': 1,
   'valeur_fonciere': 11.626263078808801},
  {'year_mutation': 2023,
   'month_mutation': 2,

In [11]:
np.expm1(df_2.valeur_fonciere).max()

2565000.0000000005

In [4]:
# Nombre total de doublons
num_duplicates = df_1.duplicated().sum()
print(f"Nombre total de lignes dupliquées : {num_duplicates}")

Nombre total de lignes dupliquées : 0


In [5]:
query = f"""
        SELECT *
        FROM `{GCP_PROJECT}`.{BQ_DATASET}.{DATA_YEAR}
    """
load_data_to_bq(
        df_2,
        gcp_project=GCP_PROJECT,
        bq_dataset=BQ_DATASET,
        table=f'clean_data_{DATA_YEAR}',
        truncate=False
    )


Save data to BigQuery @ wagon-bootcamp-429817.projimmo_datas.clean_data_ALL...:

Append wagon-bootcamp-429817.projimmo_datas.clean_data_ALL (74729 rows)
✅ Data saved to bigquery, with shape (74729, 41)


In [6]:
df_0.head(20)

,valeur_fonciere,type_de_voie,nombre_de_lots,code_type_local,surface_reelle_bati,nombre_pieces_principales,surface_terrain,month_mutation,year_mutation,somme_surface_carrez,departement
654,12.345839,RUE,1,2,95.0,4.0,0.0,1,2023,97.70,13
1027,12.409018,RUE,2,2,95.0,4.0,0.0,1,2023,94.79,13
1598,11.379406,RUE,1,2,45.0,2.0,0.0,1,2023,46.87,13
2101,11.813037,AUTRE,1,2,58.0,3.0,0.0,1,2023,60.30,13
2112,11.849405,RUE,1,2,65.0,3.0,0.0,2,2023,64.20,13
2268,11.951187,RUE,1,2,93.0,3.0,0.0,1,2023,109.20,13
2564,11.289794,RUE,1,2,61.0,2.0,0.0,1,2023,67.23,13
2781,12.345839,RTE,1,2,100.0,5.0,0.0,1,2023,120.67,13
2998,11.407576,RUE,1,2,80.0,4.0,0.0,1,2023,76.65,13
3374,11.626263,RTE,1,2,48.0,3.0,0.0,1,2023,39.03,13


In [7]:
df_0.dtypes

valeur_fonciere              float64
type_de_voie                  object
nombre_de_lots                 int64
code_type_local                int64
surface_reelle_bati          float64
nombre_pieces_principales    float64
surface_terrain              float64
month_mutation                 int64
year_mutation                  int64
somme_surface_carrez         float64
departement                    int64
dtype: object